## MLFlow Tracking with Python
> - Tested on macOS Monterey version 12.1 Macbook Pro, 2.2 GHz Quad-Core Intel Core i7, Memory 16GB DDR3
> - Please go to https://github.com/maximuslee1226/mlflow for notebooks and artifacts

## Kuzushiji Character Recognition & Classification

### Background
> "Kuzushiji is a general term (including hentaigana and cursive kanji) for characters that are not used today." 

> From the Kaggle site, it is stated that Japan has millions of books and over a billion historical documents such as personal letters or diaries preserved nationwide. Most of them cannot be read by the majority of Japanese people living today because they were written in “Kuzushiji”.

> Even though Kuzushiji, a cursive writing style, had been used in Japan for over a thousand years, there are very few fluent readers of Kuzushiji today (only 0.01% of modern Japanese natives). Due to the lack of available human resources, there has been a great deal of interest in using Machine Learning to automatically recognize these historical texts and transcribe them into modern Japanese characters.

> In this notebook, the author built a deep learning model to transcribe ancient japanese letters into contemporary Japanese character

### Notebook Code 

> - Built hand-written characters recognition system using convolutional neural networks
> - Used MLflow for tracking experimetns 

![Fig1-1: Kuzushiji Characters](http://static.mxbi.net/umgy001-010-smallannomasked.jpg)

### References
> - DHQ: Ditial Humanities Quarterly: The Kuzushiji Project
> - Kuzushiji-MNIST dataset on Kaggle

In [14]:
import os
import sys
import shutil

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [15]:
import torch
import torch.nn.functional as F
import torch.nn            as nn
import torch.optim         as optim
from torch.utils.data import DataLoader
from torchvision      import datasets, transforms
from torch.autograd   import Variable

import mlflow              as mf
import mlflow.pytorch

### Hyper Parameters

In [16]:
args = {'batch_size': 4,       
        'test_batch_size': 4,       
        'epochs': 45,       
        'seed': 32,      
        'learning_rate': 0.001,    
        'momentum': 0.5}     

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Device is now set to {}'.format(device))

Device is now set to cpu


### Convolutional Neural network 

In [17]:
class cnn(nn.Module):
    def __init__(self):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(1, 10,  kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

### Data and Dataloader Preparation

In [18]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.KMNIST('../data', train=True,  download=True, transform=transform)
test_dataset = datasets.KMNIST('../data', train=False, download=True, transform=transform)

In [19]:
train_loader = DataLoader(train_dataset, batch_size=args['batch_size'], shuffle=True, num_workers=1)
test_loader = DataLoader(test_dataset,  batch_size=args['test_batch_size'], shuffle=True, num_workers=1)


### Model and Optimizer

In [20]:
model = cnn().to(device)
optimizer = optim.SGD(model.parameters(), lr=args['learning_rate'], momentum=args['momentum'])

### Training and Testing functions

In [21]:
def train(epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 1000  == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.2f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))
            step = epoch * len(train_loader) + batch_idx
            mf.log_metric('train_loss',  loss.data.item())
            
def test(epoch):
    model.eval()
    test_loss = 0
    correct   = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').data.item() 
            pred = output.data.max(1)[1] 
            correct += pred.eq(target.data).cpu().sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.0 * correct / len(test_loader.dataset)
    
    print('\nAverage loss for Test dataset: {:.2f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))
        
    mf.log_metric('Test loss',     test_loss)
    mf.log_metric('Test Accuracy', test_accuracy)

In [22]:
model.train()

cnn(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)

### MLflow Experiment

In [23]:
mf.set_tracking_uri('http://127.0.0.1:5000')

In [24]:
experiment_id = mf.set_experiment("Kuzushiji test")

In [25]:
torch.manual_seed(args["seed"])

In [ ]:
with mf.start_run():
    
    mf_output_dir   = 'tmp/'
    os.makedirs(mf_output_dir, exist_ok=True)
    mf.log_params(args)

    for epoch in range(1, args['epochs'] + 1):
        train(epoch)
        test(epoch)
    
    mf.pytorch.log_model(model, "model")
    mf.log_artifacts(mf_output_dir)
    shutil.rmtree(mf_output_dir)

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.49
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.47
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.25
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.58
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.86
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.27
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.32
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.45
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.73
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.60
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.70
Train Epoch: 1 [44000/60000 (73%)]	Loss: 0.16
Train Epoch: 1 [48000/60000 (80%)]	Loss: 0.30
Train Epoch: 1 [52000/60000 (87%)]	Loss: 0.17
Train Epoch: 1 [56000/60000 (93%)]	Loss: 0.78

Average loss for Test dataset: 0.67, Accuracy: 5363/10000 (54%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.91
Train Epoch: 2 [4000/60000 (7%)]	Loss: 0.09
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.87
Train Epoch: 2 [12000/60000 (20%)]	Loss: 1.04
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.61
Train Epoch: 2 [20000/60000 (3

Train Epoch: 12 [0/60000 (0%)]	Loss: 0.09
Train Epoch: 12 [4000/60000 (7%)]	Loss: 0.41
Train Epoch: 12 [8000/60000 (13%)]	Loss: 0.18
Train Epoch: 12 [12000/60000 (20%)]	Loss: 0.24
Train Epoch: 12 [16000/60000 (27%)]	Loss: 0.68
Train Epoch: 12 [20000/60000 (33%)]	Loss: 0.42
Train Epoch: 12 [24000/60000 (40%)]	Loss: 0.23
Train Epoch: 12 [28000/60000 (47%)]	Loss: 0.11
Train Epoch: 12 [32000/60000 (53%)]	Loss: 0.35
Train Epoch: 12 [36000/60000 (60%)]	Loss: 1.00
Train Epoch: 12 [40000/60000 (67%)]	Loss: 0.35
Train Epoch: 12 [44000/60000 (73%)]	Loss: 0.60
Train Epoch: 12 [48000/60000 (80%)]	Loss: 0.06
Train Epoch: 12 [52000/60000 (87%)]	Loss: 0.55
Train Epoch: 12 [56000/60000 (93%)]	Loss: 0.59

Average loss for Test dataset: 0.41, Accuracy: 6364/10000 (64%)

Train Epoch: 13 [0/60000 (0%)]	Loss: 0.55
Train Epoch: 13 [4000/60000 (7%)]	Loss: 0.89
Train Epoch: 13 [8000/60000 (13%)]	Loss: 0.45
Train Epoch: 13 [12000/60000 (20%)]	Loss: 0.37
Train Epoch: 13 [16000/60000 (27%)]	Loss: 0.11
Train Epoc